In [1]:
import csv
from heapq import heapify, heappush, heappop

In [2]:
class MempoolTransaction():
    """Save each transaction as an object"""
    def __init__(self, txid, fee, weight, parents):
        self.txid = txid
        self.fee = int(fee)
        self.weight= int(weight)
        if len(parents)==0:
            self.parents =[]
        else:
            self.parents= parents.split('\n')[0].split(';')

In [3]:
def parse_mempool_csv():
    """Parse the CSV file and return a list of MempoolTransactions."""
    data= open('mempool.csv', 'r').readlines()[1:]
    return([MempoolTransaction(*line.strip().split(',')) for line in data])

In [4]:
class HeapSolution():
    
    def __init__(self):
        self.transactions= parse_mempool_csv()    
        self.heap= self.createHeap()
        self.blockWeight =0
        self.visited= set()
        self.blockTransactions= []
        self.suspendedTransactions= []      

    def createHeap(self,):
        heap = [] 
        heapify(heap) 

        for entry in self.transactions:
            heappush(heap, (-1*entry.fee, entry.txid, entry.weight, entry.parents))

        return heap
    
    def pull_suspendedTransactions(self,):
        for entry in self.suspendedTransactions:
            heappush(self.heap, (entry.fee, entry.txid, entry.weight, entry.parents))    
    
    def get_appropriateTransactions(self, ):
        while self.heap and self.blockWeight <=400000:
            entry= heappop(self.heap)
            fee, txid, weight, parents= -1*entry[0], entry[1], entry[2], entry[3]
            if len(parents) ==0:
#                 print ("No Parents")
                self.blockTransactions.append((fee, weight, txid))
                self.visited.add(txid)
                self.blockWeight +=weight
                self.pull_suspendedTransactions()

            else:
                isSuspended= False
                for parent in parents:
                    if parent not in self.visited:
#                         print ("Parent unvisited")
                        self.suspendedTransactions.append(entry)
                        isSuspended= True
                        continue
                if not isSuspended:
#                     print ("All parent visited")
                    self.blockTransactions.append(txid)
                    self.visited.add(txid)
                    self.blockWeight +=weight
                    self.pull_suspendedTransactions()
        
        profit =0
        for row in self.blockTransactions:
            profit += row[0]
            
        return profit

In [5]:
class KnapsackSolution():
    def __init__(self):
        self.transactions= parse_mempool_csv()    
        self.capacity= 40000
        self.n= len(self.transactions)
        self.blockTransactions= []
        
    def get_appropriateTransactions(self, ):     
        
        K = [[0 for w in range(self.capacity + 1)]
            for i in range(self.n + 1)]
             
        for i in range(self.n + 1):
            for w in range(self.capacity + 1):
                if i == 0 or w == 0:
                    K[i][w] = 0
                elif self.transactions[i - 1].weight <= w:
                    K[i][w] = max(self.transactions[i - 1].fee
                      + K[i - 1][w - self.transactions[i - 1].weight],
                                   K[i - 1][w])
                else:
                    K[i][w] = K[i - 1][w]
        
        
        profit = K[self.n][self.capacity]
        res =profit

        w = self.capacity
        for i in range(self.n, 0, -1):
            if res <= 0:
                break
            if res == K[i - 1][w]:
                continue
            else:

                # This item is included.
                self.blockTransactions.append(self.transactions[i-1].txid)

                # Since this weight is included
                # its value is deducted
                res = res - self.transactions[i - 1].fee
                w = w - self.transactions[i - 1].weight
                
        for row in self.blockTransactions:
            print(row)
        return profit
        

In [6]:
if __name__ == "__main__":
#     hsObj= HeapSolution()
#     print ("Profit from Heap Solution: ",hsObj.get_appropriateTransactions())
    ksObj= KnapsackSolution()
    print ("Profit from Knapsack Solution: ", ksObj.get_appropriateTransactions())

(19440, 900, 'e4f3132c345739b91e630861da990b697f7f603bd7547da320abc5282a9af65b')
(19440, 896, 'fe1360b0e758f8fa21781fd352213faf3676400a9415a319680664243b08cad1')
(100000, 1148, 'c3fef085fca34891e6456489d840ab68139b24857eb1f925b943066ebb988732')
(45075, 2400, 'fac0417aafa46ea002ed3e04fc38087b45aca6a15a47bd4e5026e1e6cefa7967')
(104400, 2084, '0c8ebf9c75f63b7e5ff176e2937f24c694aa6b3bde0e59b5647983bbb7dd38d6')
(54675, 904, 'b8894fbe99628c253fa93cf178679727e117d04fea5e5079de002548a0dd6511')
(107775, 1660, '87784075804f10dad1f815de867dde2875e73a13da798c317fcddd75e03efc95')
(59326, 1364, '6a709ddadfcf13b2e302cf0f75163538b0273923cc55fccc158f7466abebc1a9')
(90000, 896, '826c80c43044cc00bebdf021a42dca6946591f02710e4e6da58c094be8e62d00')
(19440, 892, '71135d5eea18edfaba370a457bc5135fdf97639b207dc682f940939743c9b32e')
(16600, 664, '24e784e1c9373f72dcc8a6923bf5558ba7b544158b56e097ec393e8695b19a5e')
(40200, 2668, '7f264c468f624b62071dbcb531de5af722b327d5b098f426314622340cf17512')
(194166, 3852, '3bf